In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# https://www.kaggle.com/competitions/nlp-getting-started

import numpy as np 
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate

train_dataset = load_dataset('csv', data_files='train.csv')
test_dataset = load_dataset('csv', data_files='test.csv')

dataset = DatasetDict({
    'train': train_dataset['train'],
    'test': test_dataset['train']
})

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

dataset['train'] = dataset['train'].rename_column("target", "labels")
dataset = dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
# metrics.compute(predictions=[0, 1, 0], references=[0, 1, 1])

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=metrics,
)

trainer.train()




#TODO: may need to rename the label to label https://huggingface.co/docs/datasets/process#rename
#TODO: make sure label is cast correctly, and do so if not already https://huggingface.co/docs/datasets/process#cast

#TODO: make sure to shuffle the dataset when training (and take account of this in terms of IDS and predictions).  shuffling makes sense because the dataset is sorted, and BERT would be finetuned by SGD and order would have an effect.
    # https://huggingface.co/docs/datasets/process#shuffle
    # " as soon as your Dataset has an indices mapping, the speed can become 10x slower. This is because there is an extra step to get the row index to read using the indices mapping, and most importantly, you aren’t reading contiguous chunks of data anymore. To restore the speed, you’d need to rewrite the entire dataset on your disk again using Dataset.flatten_indices(), which removes the indices mapping."
    # Note: no need to shuffle during testing
    # QSTN: reshuffle per epoch?


#TODO: Load the automodel or whatev through pipeline.  Identify which model it uses by default for text-classification and read the docs on it.  I'm guessing it's BERT.

#TODO: Preprocess (tokenize) the data.  this might be handled by pipeline dknow.
    # https://huggingface.co/docs/datasets/nlp_process
    # def tokenization(example):
    #     return tokenizer(example["text"])

    # dataset = dataset.map(tokenization, batched=True)


#TODO: see if including the fields beyond the text of the tweet impacts F1 score or not.  Let's do first pass without that stuff and see how leaderboard goes.



#TODO: score on F1
    # See https://huggingface.co/docs/evaluate/index


#TODO: write results to CSV
# For each ID in the test set, you must predict 1 if the tweet is describing a real disaster, and 0 otherwise. The file should contain a header and have the following format:
# id,target
# 0,0
# 2,0
# 3,1
# 9,0
# 11,0
#sample_submission.to_csv("submission.csv", index=False)

#TODO: consider if stratification during training would help with F1.
#TODO: look at other ppl's notebooks that used HF Transformers and see how I can improve score.
